# Data Exploration & Process to define Primary Key

In [1]:
import pandas as pd

In [3]:
# Look at the data: columns, types, etc.

df = pd.read_csv('./event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


## Question 1 :

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [31]:
print("--> The user named 'Ava Robison' has {} unique session_id\n".format(len(df.loc[(df['lastName']=='Robinson') & (df['firstName']=='Ava')]['sessionId'].unique())))
print("--> The user_id '50' corresponding to 'Ava Robinson' living in CT has {} unique session_id\n".format(len(df.loc[df['userId']==50]['sessionId'].unique())))
print("--> The user_id '50' corresponding to 'Ava Robinson' living in London has {} unique session_id\n".format(len(df.loc[df['userId']==13]['sessionId'].unique())))

df.loc[df['sessionId']==338]

--> The user named 'Ava Robison' has 26 unique session_id

--> The user_id '50' corresponding to 'Ava Robinson' living in CT has 23 unique session_id

--> The user_id '50' corresponding to 'Ava Robinson' living in London has 3 unique session_id



,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
441,Pixies,Ava,F,1,Robinson,89.36444,free,"New Haven-Milford, CT",338,Build High,50
442,The Roots / Jack Davey,Ava,F,2,Robinson,155.95057,free,"New Haven-Milford, CT",338,Atonement,50
443,Mike And The Mechanics,Ava,F,3,Robinson,275.12118,free,"New Haven-Milford, CT",338,A Beggar On A Beach Of Gold,50
444,Faithless,Ava,F,4,Robinson,495.30730,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


**General Information**
1. The same user can have multiple session_id
2. Two user (user_id) can have the same name (first+Last)
3. While the 2 users in this exploration don't live in the same city, it is a possibility to have 2 user with the same name living in the same city

**Primary Key** 

So far, by looking at the data, we can see that:
1. Session_id is linked to a specific user (first name, gender, last name, level) which make it perfect for a Partition Key. 
2. Eeach session_id is composed of rows about the songs listen by the user (artist, song name, duration)
3. Each song in a session is uniquely identify by the incremential variable itemInSession. 

Partition Key should be : sessionId

Clustering Key should be : None

In [18]:
df.loc[(df['lastName']=='Robinson') & (df['firstName']=='Ava')]['sessionId'].unique()

array([ 940,  979,  313,  338,  343,  108,  360,  824,  623,  653,  667,
        422, 1011,  598,  211,  694,  836,  156,  207,  988,  908,  430,
       1050,  538,  578,  669])